In [1]:
import numpy as np
import pandas as pd

In [2]:
data=pd.read_csv('../Data/wine.csv')
data.head()

,T,A,B,C,D,E,F,G,H,I,J,K,L,M
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [3]:
target=data['T'].unique()

d={}
for i in range(len(target)):
    d[target[i]]=i

target=np.asarray(data['T'])
data=np.asarray(data.drop('T',axis=1))

for i in range(len(target)):
    target[i]=d[target[i]]

print(d)

{1: 0, 2: 1, 3: 2}


In [4]:
def initialise_weights(TotalW,TotalB,a,b):
    W=np.zeros((a,b))+np.random.randn(a,b)
    B=np.zeros((b,1))+np.random.randn(b,1)
    TotalW.append(W)
    TotalB.append(B)

    return TotalW,TotalB

def normalise_scale(N,a,b):
    minN=min(N)
    maxN=max(N)
    for i in range(len(N)):
        N[i]=((N[i]-minN)/(maxN-minN))*(a-b)+b
    return N
    
def cost(true,pred):
    return np.sum((true-pred)**2)
    
def sigmoid(z):
    return 1/(1+np.exp(-z))

def minibatch_gradient_descent(TotalW,TotalB,data,l,max_iterations,learning_rate,batch_size):
    cnt=0
    flag=0
    while(cnt<max_iterations):
        cnt+=1
        
        TotalgradW=[]
        TotalgradB=[]

        for i in range(len(data)):  
            pred,TotalI=forward_propagation(TotalW,TotalB,data[i].reshape(len(data[i]),1),l)

            c=cost(data[i].reshape(len(data[i]),1),pred)
            
            gradW,gradB=backward_propagation(data[i].reshape(len(data[i]),1),pred,TotalI,l)
            
            if(i==0 or flag==1):
                TotalgradW=gradW
                TotalgradB=gradB
                flag=0
            else:
                for j in range(len(gradW)):
                    TotalgradW[j]+=gradW[j]
                    TotalgradB[j]+=gradB[j]
            
            if((i+1)%batch_size==0):

                for j in range(len(TotalW)):
                    TotalW[j]-=learning_rate*TotalgradW[j]
                    TotalB[j]-=learning_rate*TotalgradB[j]
                flag=1
                    
                print("loss ---- ",c)
                print(pred)
                print(data[i].reshape(len(data[i]),1))
                print()

def forward_propagation(TotalW,TotalB,data,l):
    TotalI=[]
    TotalI.append(data)
    for i in range(l-1):
        data=sigmoid(np.dot(TotalW[i].T,data)+TotalB[i])
        TotalI.insert(0,data)
    
    data=(np.dot((TotalW[i+1].T),data)+TotalB[i+1])
    return data,TotalI
    

def backward_propagation(one_hot,pred,TotalI,l):
    TotalgradW=[]
    TotalgradB=[]
    
    grad_final_layer=-2*(one_hot-pred)
    grad_hidden_layer=grad_final_layer
    
    for i in range(l):
    
        gradW=np.dot(TotalI[i],grad_hidden_layer.T)
        gradB=grad_hidden_layer
        
        grad_hidden_layer=np.dot(TotalW[l-1-i],grad_hidden_layer)
        grad_hidden_layer*=TotalI[i]*(1-TotalI[i])
        
        TotalgradW.insert(0,gradW)
        TotalgradB.insert(0,gradB)
    
    return TotalgradW,TotalgradB
    


In [5]:
TotalW=[]
TotalB=[]

input_neurons=len(data[0])
output_neurons=len(d)
max_iterations=1000

#Parameter1
TotalW,TotalB=initialise_weights(TotalW,TotalB,input_neurons,12)
#Parameter2
TotalW,TotalB=initialise_weights(TotalW,TotalB,12,11)
#Parameter3
TotalW,TotalB=initialise_weights(TotalW,TotalB,11,10)
#Parameter4
TotalW,TotalB=initialise_weights(TotalW,TotalB,10,11)
#Parameter5
TotalW,TotalB=initialise_weights(TotalW,TotalB,11,12)
#Parameter6
TotalW,TotalB=initialise_weights(TotalW,TotalB,12,input_neurons)



#Total Layers
l=len(TotalW)

#learning rate
learning_rate=0.004

for i in range(len(data[0])):
    data[:,i]=normalise_scale(data[:,i],0.999,.001)

minibatch_gradient_descent(TotalW,TotalB,data,l,max_iterations,learning_rate,32)

loss ----  41.07766352693912
[[-1.42546545]
 [ 1.36673411]
 [ 0.21001389]
 [ 4.03421411]
 [ 0.90814696]
 [ 0.97086845]
 [ 2.82845591]
 [ 0.27599252]
 [-2.83709594]
 [-0.11571449]
 [ 1.13736315]
 [-1.57554131]
 [ 0.24431363]]
[[0.67071053]
 [0.18245455]
 [0.53468984]
 [0.43826804]
 [0.39152174]
 [0.64797931]
 [0.60106329]
 [0.1704717 ]
 [0.48583281]
 [0.47956314]
 [0.49594309]
 [0.5895641 ]
 [0.88154636]]

loss ----  7.518477253434384
[[ 1.27429955]
 [ 0.95201375]
 [ 1.61748756]
 [ 0.46033659]
 [-0.40557617]
 [ 1.63942383]
 [ 0.03101034]
 [-0.96287209]
 [ 0.88824866]
 [ 0.44396897]
 [-0.08911291]
 [-0.31974398]
 [ 0.85878827]]
[[0.35292632]
 [0.07792095]
 [0.42795187]
 [0.43312371]
 [0.18541304]
 [0.86822759]
 [0.58211392]
 [0.11398113]
 [0.46064669]
 [0.27093686]
 [0.60142276]
 [0.58590842]
 [0.10208131]]

loss ----  5.426925719384539
[[ 0.7421449 ]
 [-0.05184824]
 [-0.10112116]
 [ 0.11504495]
 [ 1.55453261]
 [-0.42740615]
 [ 0.97831857]
 [-0.63886999]
 [ 0.23106053]
 [ 0.83649592]
 [ 

loss ----  0.9894438657833914
[[0.39197642]
 [0.25040612]
 [0.54973408]
 [0.48868677]
 [0.24702129]
 [0.51165528]
 [0.40007209]
 [0.41913805]
 [0.44711654]
 [0.21161692]
 [0.40101855]
 [0.55484643]
 [0.24131233]]
[[0.64444737]
 [0.18442688]
 [0.68412299]
 [0.61317526]
 [0.2071087 ]
 [0.55850345]
 [0.16101688]
 [0.73537736]
 [0.59287382]
 [0.89255802]
 [0.07402439]
 [0.18743956]
 [0.24444936]]

loss ----  0.9422740563760171
[[0.49327067]
 [0.43115616]
 [0.59195894]
 [0.48920014]
 [0.22672539]
 [0.47752085]
 [0.26606614]
 [0.49267815]
 [0.32915002]
 [0.34370649]
 [0.21851969]
 [0.33028794]
 [0.26218022]]
[[0.67071053]
 [0.18245455]
 [0.53468984]
 [0.43826804]
 [0.39152174]
 [0.64797931]
 [0.60106329]
 [0.1704717 ]
 [0.48583281]
 [0.47956314]
 [0.49594309]
 [0.5895641 ]
 [0.88154636]]

loss ----  0.6138930223971825
[[0.57384365]
 [0.28686912]
 [0.48080608]
 [0.42862163]
 [0.43143019]
 [0.39745169]
 [0.41688382]
 [0.4081553 ]
 [0.35785328]
 [0.3241049 ]
 [0.49241062]
 [0.59541189]
 [0.4139

loss ----  0.6079821786699964
[[0.57044403]
 [0.28541512]
 [0.48383619]
 [0.4308934 ]
 [0.42733282]
 [0.39904464]
 [0.41624498]
 [0.41046803]
 [0.36075642]
 [0.32122153]
 [0.49035967]
 [0.59442327]
 [0.41110791]]
[[0.35292632]
 [0.07792095]
 [0.42795187]
 [0.43312371]
 [0.18541304]
 [0.86822759]
 [0.58211392]
 [0.11398113]
 [0.46064669]
 [0.27093686]
 [0.60142276]
 [0.58590842]
 [0.10208131]]

loss ----  0.8163923187192043
[[0.61557641]
 [0.29399213]
 [0.5256238 ]
 [0.39263221]
 [0.36396179]
 [0.54437784]
 [0.43786039]
 [0.35552208]
 [0.38880872]
 [0.32600687]
 [0.43496836]
 [0.56077035]
 [0.46119427]]
[[0.37918947]
 [0.1548419 ]
 [0.44929947]
 [0.43312371]
 [0.999     ]
 [0.52408966]
 [0.40735865]
 [0.35877358]
 [0.90455205]
 [0.11340273]
 [0.55273984]
 [0.49817216]
 [0.47010271]]

loss ----  0.7299965571985492
[[0.47543578]
 [0.1857639 ]
 [0.47778016]
 [0.44842605]
 [0.37694873]
 [0.42591984]
 [0.41296927]
 [0.37532786]
 [0.41074459]
 [0.24908172]
 [0.53467213]
 [0.60343922]
 [0.3352

loss ----  0.9783859139445586
[[0.39962042]
 [0.25686776]
 [0.5471557 ]
 [0.48291077]
 [0.25167111]
 [0.51187959]
 [0.39982036]
 [0.41963674]
 [0.43942853]
 [0.21885704]
 [0.39964819]
 [0.55231975]
 [0.2462977 ]]
[[0.64444737]
 [0.18442688]
 [0.68412299]
 [0.61317526]
 [0.2071087 ]
 [0.55850345]
 [0.16101688]
 [0.73537736]
 [0.59287382]
 [0.89255802]
 [0.07402439]
 [0.18743956]
 [0.24444936]]

loss ----  0.9160718674078334
[[0.49166817]
 [0.42073552]
 [0.59171426]
 [0.49311128]
 [0.22476384]
 [0.48130617]
 [0.26856412]
 [0.47947045]
 [0.33862764]
 [0.33870548]
 [0.22446651]
 [0.33491812]
 [0.26567308]]
[[0.67071053]
 [0.18245455]
 [0.53468984]
 [0.43826804]
 [0.39152174]
 [0.64797931]
 [0.60106329]
 [0.1704717 ]
 [0.48583281]
 [0.47956314]
 [0.49594309]
 [0.5895641 ]
 [0.88154636]]

loss ----  0.6010466063316295
[[0.56650777]
 [0.2833341 ]
 [0.48678046]
 [0.43392876]
 [0.42302445]
 [0.40043207]
 [0.41541873]
 [0.41203027]
 [0.36437638]
 [0.31752334]
 [0.48851846]
 [0.59346496]
 [0.4080

loss ----  0.5954967583359905
[[0.56333707]
 [0.28137368]
 [0.48870936]
 [0.43662365]
 [0.4199442 ]
 [0.40116989]
 [0.41476408]
 [0.412582  ]
 [0.3674293 ]
 [0.31428418]
 [0.48745527]
 [0.59289913]
 [0.40573441]]
[[0.35292632]
 [0.07792095]
 [0.42795187]
 [0.43312371]
 [0.18541304]
 [0.86822759]
 [0.58211392]
 [0.11398113]
 [0.46064669]
 [0.27093686]
 [0.60142276]
 [0.58590842]
 [0.10208131]]

loss ----  0.8102813932144434
[[0.61160806]
 [0.29159068]
 [0.52401675]
 [0.39512537]
 [0.36517131]
 [0.53934643]
 [0.43792368]
 [0.35577526]
 [0.38959751]
 [0.32439164]
 [0.435673  ]
 [0.56186633]
 [0.45724444]]
[[0.37918947]
 [0.1548419 ]
 [0.44929947]
 [0.43312371]
 [0.999     ]
 [0.52408966]
 [0.40735865]
 [0.35877358]
 [0.90455205]
 [0.11340273]
 [0.55273984]
 [0.49817216]
 [0.47010271]]

loss ----  0.7320651077205425
[[0.47678341]
 [0.18901714]
 [0.47476284]
 [0.44643531]
 [0.38174916]
 [0.42207732]
 [0.41371332]
 [0.38079963]
 [0.40599048]
 [0.24973086]
 [0.5375103 ]
 [0.60617869]
 [0.3358

...

loss ----  0.5055528762232508
[[0.64055128]
 [0.53093129]
 [0.62255545]
 [0.47631063]
 [0.32133547]
 [0.36180235]
 [0.23921379]
 [0.65491192]
 [0.2924204 ]
 [0.48271217]
 [0.20154213]
 [0.2631669 ]
 [0.38002255]]
[[0.64444737]
 [0.18442688]
 [0.68412299]
 [0.61317526]
 [0.2071087 ]
 [0.55850345]
 [0.16101688]
 [0.73537736]
 [0.59287382]
 [0.89255802]
 [0.07402439]
 [0.18743956]
 [0.24444936]]

loss ----  0.10016425864215175
[[0.78110892]
 [0.25101238]
 [0.59491856]
 [0.34628611]
 [0.45211482]
 [0.68638583]
 [0.56765001]
 [0.18623316]
 [0.51041167]
 [0.45135194]
 [0.4387664 ]
 [0.63624852]
 [0.64107173]]
[[0.67071053]
 [0.18245455]
 [0.53468984]
 [0.43826804]
 [0.39152174]
 [0.64797931]
 [0.60106329]
 [0.1704717 ]
 [0.48583281]
 [0.47956314]
 [0.49594309]
 [0.5895641 ]
 [0.88154636]]

loss ----  0.2449778822823032
[[0.44496376]
 [0.04636277]
 [0.45576948]
 [0.42689219]
 [0.3855675 ]
 [0.5637937 ]
 [0.50174008]
 [0.06213191]
 [0.46269968]
 [0.18510461]
 [0.60812873]
 [0.71146063]
 [0.365

loss ----  0.24474414455024424
[[0.44478534]
 [0.04643436]
 [0.45572074]
 [0.42656643]
 [0.38558205]
 [0.56379434]
 [0.50188781]
 [0.06225451]
 [0.46270957]
 [0.18503525]
 [0.60780253]
 [0.71139074]
 [0.36565595]]
[[0.35292632]
 [0.07792095]
 [0.42795187]
 [0.43312371]
 [0.18541304]
 [0.86822759]
 [0.58211392]
 [0.11398113]
 [0.46064669]
 [0.27093686]
 [0.60142276]
 [0.58590842]
 [0.10208131]]

loss ----  0.7196288662184763
[[0.4900029 ]
 [0.09360745]
 [0.46376109]
 [0.41337331]
 [0.39720293]
 [0.53736866]
 [0.45968112]
 [0.10396785]
 [0.41730574]
 [0.23232581]
 [0.55635938]
 [0.61419073]
 [0.38079618]]
[[0.37918947]
 [0.1548419 ]
 [0.44929947]
 [0.43312371]
 [0.999     ]
 [0.52408966]
 [0.40735865]
 [0.35877358]
 [0.90455205]
 [0.11340273]
 [0.55273984]
 [0.49817216]
 [0.47010271]]

loss ----  0.30015018873560506
[[0.32137639]
 [0.34359718]
 [0.51903209]
 [0.50744595]
 [0.18616284]
 [0.36023648]
 [0.28168348]
 [0.70719097]
 [0.3320881 ]
 [0.17138165]
 [0.38226876]
 [0.48734564]
 [0.12

loss ----  0.2997953012758082
[[0.32141358]
 [0.34418182]
 [0.51924687]
 [0.50768496]
 [0.18572161]
 [0.36066133]
 [0.28193286]
 [0.70766455]
 [0.332284  ]
 [0.17122603]
 [0.382254  ]
 [0.48801339]
 [0.12129237]]
[[0.2006    ]
 [0.27515415]
 [0.75883957]
 [0.92183505]
 [0.23965217]
 [0.39675862]
 [0.40104219]
 [0.84835849]
 [0.42601577]
 [0.14746416]
 [0.39857724]
 [0.42871429]
 [0.13482596]]

loss ----  0.5048608883680114
[[0.64184789]
 [0.53027708]
 [0.62275865]
 [0.47585956]
 [0.32158594]
 [0.3621119 ]
 [0.23905441]
 [0.6551287 ]
 [0.2925228 ]
 [0.48360278]
 [0.20223931]
 [0.26315771]
 [0.3815215 ]]
[[0.64444737]
 [0.18442688]
 [0.68412299]
 [0.61317526]
 [0.2071087 ]
 [0.55850345]
 [0.16101688]
 [0.73537736]
 [0.59287382]
 [0.89255802]
 [0.07402439]
 [0.18743956]
 [0.24444936]]

loss ----  0.09998935835682204
[[0.78139466]
 [0.25036148]
 [0.59500928]
 [0.34595812]
 [0.45244252]
 [0.68615045]
 [0.56742036]
 [0.18688175]
 [0.5103072 ]
 [0.45169706]
 [0.43891034]
 [0.63559948]
 [0.641

loss ----  0.7192932621286537
[[0.49060175]
 [0.09376513]
 [0.46385466]
 [0.41242067]
 [0.39764341]
 [0.53756987]
 [0.46004777]
 [0.10387103]
 [0.41729417]
 [0.23282571]
 [0.55560308]
 [0.6136849 ]
 [0.38114588]]
[[0.37918947]
 [0.1548419 ]
 [0.44929947]
 [0.43312371]
 [0.999     ]
 [0.52408966]
 [0.40735865]
 [0.35877358]
 [0.90455205]
 [0.11340273]
 [0.55273984]
 [0.49817216]
 [0.47010271]]

loss ----  0.2993658638996393
[[0.32145887]
 [0.34487887]
 [0.51951123]
 [0.50797885]
 [0.18519795]
 [0.3611682 ]
 [0.28222912]
 [0.70822504]
 [0.33251876]
 [0.17104115]
 [0.38223801]
 [0.48880714]
 [0.12137637]]
[[0.2006    ]
 [0.27515415]
 [0.75883957]
 [0.92183505]
 [0.23965217]
 [0.39675862]
 [0.40104219]
 [0.84835849]
 [0.42601577]
 [0.14746416]
 [0.39857724]
 [0.42871429]
 [0.13482596]]

loss ----  0.5044913080950157
[[0.64253656]
 [0.52992517]
 [0.62286903]
 [0.4756213 ]
 [0.32171221]
 [0.36227654]
 [0.23896249]
 [0.65525244]
 [0.29257492]
 [0.48407583]
 [0.20261319]
 [0.26314675]
 [0.3823

loss ----  0.24398627808051992
[[0.44422118]
 [0.04667517]
 [0.45553643]
 [0.42543735]
 [0.38562339]
 [0.56380252]
 [0.50238654]
 [0.06269538]
 [0.46275078]
 [0.18483043]
 [0.60671462]
 [0.7111851 ]
 [0.36470035]]
[[0.35292632]
 [0.07792095]
 [0.42795187]
 [0.43312371]
 [0.18541304]
 [0.86822759]
 [0.58211392]
 [0.11398113]
 [0.46064669]
 [0.27093686]
 [0.60142276]
 [0.58590842]
 [0.10208131]]

loss ----  0.7191178874631198
[[0.49092123]
 [0.09384465]
 [0.46389497]
 [0.41189997]
 [0.39787549]
 [0.5376727 ]
 [0.46023925]
 [0.10382903]
 [0.41728571]
 [0.23309432]
 [0.55519937]
 [0.6134149 ]
 [0.3813333 ]]
[[0.37918947]
 [0.1548419 ]
 [0.44929947]
 [0.43312371]
 [0.999     ]
 [0.52408966]
 [0.40735865]
 [0.35877358]
 [0.90455205]
 [0.11340273]
 [0.55273984]
 [0.49817216]
 [0.47010271]]

loss ----  0.29893253052060625
[[0.32150483]
 [0.34557111]
 [0.51978278]
 [0.50828029]
 [0.18468053]
 [0.36167194]
 [0.2825222 ]
 [0.70877723]
 [0.33275318]
 [0.17085822]
 [0.38222384]
 [0.48959273]
 [0.12

In [6]:
a=10
p,x=forward_propagation(TotalW,TotalB,data[i].reshape(len(data[a]),1),l)
print(p,data[a].reshape(len(data[a]),1))
c=cost(data[i].reshape(len(data[a]),1),p)
print(c)

[[0.71462141]
 [0.27965261]
 [0.58609365]
 [0.36266526]
 [0.40492924]
 [0.66515252]
 [0.55395187]
 [0.28680525]
 [0.50372141]
 [0.40563147]
 [0.42491095]
 [0.65219239]
 [0.56916274]] [[0.80727895]
 [0.28107115]
 [0.50266845]
 [0.38168041]
 [0.38067391]
 [0.67895172]
 [0.6284346 ]
 [0.1704717 ]
 [0.6212082 ]
 [0.38163652]
 [0.62576423]
 [0.69557875]
 [0.87798716]]
0.11994375186795458
